In [ ]:
from abc import ABCMeta
import argparse
import collections
from collections import defaultdict
import json
import math
import os
import random
import pickle
import sys
from tqdm import tqdm, trange
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.nn import CrossEntropyLoss
from pathlib import Path
from collections import Counter

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering

from pytorch_pretrained_bert.optimization import BertAdam
from utils.ConfigLogger import config_logger
from utils.evaluate import f1_score, exact_match_score, metric_max_over_ground_truths
from utils.BERTRandomSampler import BERTRandomSampler

PYTORCH_PRETRAINED_BERT_CACHE = Path(os.getenv('PYTORCH_PRETRAINED_BERT_CACHE',
                                               Path.home() / '.pytorch_pretrained_bert'))

from data_utils import *
from qada_utils import *

In [ ]:
# SquadExample: 원본 QA 데이터를 한 문항(질문+문맥+정답) 객체로 저장
SquadExample(
    qas_id="56be85543aeaaa14008c9063",   # 질문 ID
    question_text="When did Beyonce start becoming popular?",  # 질문
    doc_tokens=["Beyonce", "is", "an", "American", "singer", ...],  # 문맥 토큰
    orig_answer_text="in the late 1990s",  # 원래 정답 텍스트
    start_position=14,   # 문맥 토큰 기준 시작 인덱스
    end_position=17,     # 문맥 토큰 기준 끝 인덱스
    answers=["in the late 1990s", "during the late 1990s"], # 정답 후보 리스트
    q_type=None          # 질문 타입(옵션)
)

# Features: SquadExample를 BERT데 입력 가능한 텐서로 변환한 것
# [CLS] 질문 [SEP] 문맥 [SEP] 형태로 WordPiece 토큰화하고, 인덱스로 바꿔서 고정 길이 텐서(input_ids, input_mask, segment_ids)로 만든 상태.
InputFeatures(
    unique_id=1000000001,
    example_index=0,           # 어떤 example에서 나온 feature인지
    doc_span_index=0,          # 긴 문맥을 자른 조각 번호
    tokens=["[CLS]", "when", "did", "be", "##yon", "##ce", ...],
    token_to_orig_map={10: 5, 11: 6, ...},  # WordPiece ↔ 원래 토큰 매핑
    token_is_max_context={11: True, ...},   # 해당 토큰이 최적 context인지 여부
    input_ids=[101, 2043, 2106, 2022, ...], # BERT vocab ID
    input_mask=[1, 1, 1, 1, 1, 1, 0, 0, ...], # 실제 토큰=1, 패딩=0
    segment_ids=[0,0,0,0,0,1,1,1,...],       # 질문=0, 문맥=1
    start_position=25,  # 정답 시작 위치 (input_ids 기준)
    end_position=28,    # 정답 끝 위치
    q_type=None
)

example:
question_text = "When did Beyonce start becoming popular?"
doc_tokens = ["Beyonce", "is", "an", "American", ...]
answer = "in the late 1990s"

feature:
input_ids = [101, 2043, 2106, 2022, 2327, 2466, 102, 2022, 2148, ...]
input_mask = [1, 1, 1, 1, ...]
segment_ids = [0, 0, 0, 0, 0, 1, 1, 1, ...]
start_position = 25, end_position = 28